# Flask Workshop

### Australian Synchrotron - 15/12/2016

## What is Flask?

- Python library
- Server-side web micro-framework for Python
- Unopinionated
- Excellent documentation

Most importantly: A lot of fun!

## Resources

### Main
- [Main website](http://flask.pocoo.org/)
- [Author's tutorial](http://flask.pocoo.org/docs/0.11/tutorial/)

### Miguel Grinberg
- [The Mega Tutorial](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world)
- [His book](http://shop.oreilly.com/product/0636920031116.do)

### Australian Synchrotron
- [Confluence](https://confluence.synchrotron.org.au/pages/viewpage.action?pageId=7144364)
- [Talk (highly recommended for learning the basics of webservices)](https://events.synchrotron.org.au/event/35/session/0/material/0/0.pdf)

## Prerequisites

### Software
- Python 3.5
- Web Browser
- Text editor or Python IDE
- Terminal/Shell/cmd.exe
- Conda or pip virtual environment

### Python Packages (via conda install or pip install)
- Flask
- flask-bootstrap
- httpie

## Goal of this tutorial

In this tutorial we will create a little todo web application. Users will be able to create a list of todo items.

At the end, you will have the skills

- to write simple websites and webservices
- send HTTP requests to a webservice
- master the creation of dynamic text output via jinja templates

## Let's get started!

Open the text editor or Python IDE (PyCharm, Spyder, ...) of your choice.

Create an empty file called `app.py` (or any other name you fancy).

Add the following line in order to import Flask:

In [ ]:
from flask import Flask

Create an `app` object, by adding:

In [ ]:
app = Flask(__name__)

In order to run the Flask application add the following code to the end of the file

In [ ]:
if __name__ == '__main__':
    app.run()

Now, run the application by either clicking the run button in your IDE or by executing:

In [ ]:
python app.py

The application should start and tell you that it is running at http://127.0.0.1:5000/. Excellent! Open a browser and enter this URL. You will be greeted with a "404 Not Found" error.

What went wrong?

Nothing really. Flask simply doesn't know what to do with the request from the browser. So far you have the application running, but you don't have any website yet. Let's change this. Add the following between the `app` creation line and the line `if __name__ == '__main__'`:

In [ ]:
@app.route('/')
def index():
    return 'Hello Synchrotron!'

This tells Flask to execute the function `index()` when you browse to the root URL. The function then returns the string "Hello Synchrotron".

Restart the running Flask application and refresh the browser. It should show "Hello Synchrotron!". Much better!

However, it is tedious to restart the Flask application every time you change the source code. Switch the Flask app to debug mode, and it will automatically restart everytime a file is changed. Simply set the `debug` flag in the last line to `True`:

In [ ]:
app.run(debug=True)

**Task**: Now change the returned string in the `index()` method to something else, save the file and watch how the server is automatically restarted. The only thing you have to do from now on is to refresh the browser after you changed some code.

## Routing

Having a single route is nice, but not very exciting. Let's add another route:

In [ ]:
@app.route('/user/steve')
def show_user():
    return 'Hello Steve!'

**Task**: Which URL do you have to enter now in order to see the text "Hello Steve?"

### Dynamic routes

Most likely your name is not Steve, so the output doesn't make any sense. We have to make this page a bit more dynamic by using template parameters. Change the decorator to: 

In [ ]:
@app.route('/user/<username>')

And add a parameter called `name` to the function:

In [ ]:
def show_user(username):

This will store anything you enter after "/user/" in your URL into the variable `username`.

**Task**: Change the returned string such that it greets you with the name entered in the URL. Try it out by entering various names.

### Parameter types

The template parameters are not restricted to strings. You can, for example, also accept integer values and let Flask know that you only accept integers. Give this a go:

In [ ]:
@app.route('/item/<int:item_id>')
def show_item(item_id):
    return 'This is item number {}'.format(item_id)

**Task**: Try numbers, but also strings. Check what happens. There are more types available. See here: http://flask.pocoo.org/docs/0.11/quickstart/#variable-rules. Try them out.

### Request methods

Every time you enter a URL into a browser, a so-called GET HTTP request is sent to the webserver. But you are not restricted to GET requests. HTTP defines a number of methods. You can find an overview on page 4 of our [seminar talk](https://events.synchrotron.org.au/event/35/session/0/material/0/0.pdf).

Let's create a function that can be used to create a new todo item. We will use the POST method for this. The list of methods that a route accepts are specified in the `methods` parameter. Add the following function to your code:

In [ ]:
@app.route('/item/add', methods=['POST'])
def add_item():
    return 'Added item'

**Task**: Try to call this route in your browser and observe what happens.

A web browser only sends a GET request to the web server when you enter a URL. But this route is limited to POST requests!

We will have to use a different way for testing this route. We recommend the `httpie` tool, but please feel free to use `curl` or your preferred method. Using `httpie`, enter the following into a terminal or shell:

In [ ]:
http POST 127.0.0.1:5000/item/add

The reponse from http should be:

In [ ]:
HTTP/1.0 200 OK

together with the text: `Added item`.

Great! So we can send a POST request, but we are not sending any information with our request. Let's change this:

In [ ]:
http --form POST 127.0.0.1:5000/item/add username=Steve
    task='Write Nature paper'

This request sends so-called url-form-encoded data to our little webserver. Think of it as a list of key=value pairs. The idea behind the request is to tell our webserver that we want to add a new todo item with the task "Write Nature paper" and assign it to the user "Steve".

The next step is to modify our webserver to handle the data. The incoming request is available from a global request object. Let's start by importing this global request object. Modify the first line of your script to:

In [ ]:
from flask import Flask, request

Now we can access the data that has been sent with the request by using the `form` attribute of the request object. So the username and task are available from:

In [ ]:
request.form['username']

and

In [ ]:
request.form['task']

**Task**: Modify the `add_item()` function such that it returns a string containing the provided username and task. Test your code by using the `http` call from above.

Ok, so far we can send data to the webserver and the webserver can read the incoming data. But without storing the data, what is even the point of sending any data to the application?

In order to keep things simple, we will store the data into a JSON file on disk.

**Please note** this is not something you should do in a production environment with mutliple users accessing the webserver at the same time. Because while your webserver handles one request another request might come in and potentially overwrite the JSON file the first request created.

In order to make json available, add it to the `import` statements at the top of the script:

In [ ]:
from flask import Flask, request
import json

Below the `import` statement, define the location and name of the JSON file, so we have a single spot where this information is defined.

In [ ]:
JSON_FILE = 'todo_items.json'

To get started, let's create an empty JSON file that will store all our todo items. In the directory of your Python file, create a file called `todo_items.json` with the following content:

In [ ]:
{
  "items": []
}

Now we are ready to store the data. First open the existing JSON file in your `add_item()` function:

In [ ]:
@app.route('/item/add', methods=['PUT'])
def add_item():
    with open(JSON_FILE) as data_file:
        item_dict = json.load(data_file)

    return 'Added item'

This opens the file defined in `JSON_FILE` and passes a file object (`data_file`) to the `load()` function. The result is a standard Python dictionary in `item_dict`.

**Task**: Test whether it works and it can find the file.

Before we move on and start adding the incoming data to the `item_dict`, let's change the `return` statement so that it prints the content of `item_dict`. Flask has the very handy `jsonify` function built in. It takes a Python dictionary and constructs a valid JSON HTTP response. In order to make it available, add it to your `import` statements:

In [ ]:
from flask import Flask, request, jsonify
import json

Then change your return statement in `add_item()` to:

In [ ]:
return jsonify(item_dict)

**Task**: Send a request with `http` and check the output.

**Task**: Append a dictionary with the `username` and `task` from the `request.form` to the `item_dict`, such that the `item_dict` looks something like this:

In [ ]:
{
    "items": [
        {
            "task": "The task provided by the http request",
            "username": "The username provided by the http request"
        }
    ]
}

Now we are ready to save the data in `item_dict` back to disk. Add the following line before the `return` statement to the `add_item()` function:

In [ ]:
with open(JSON_FILE, 'w') as data_file:
    json.dump(item_dict, data_file)

**Task**: Call the POST http request a couple of times with different values for `username` and `task` and check its output. Also, have a look at the JSON file on disk.

## Templates

Let's return to our webpage, the part that our users see. So far we only print simple, static text. Next, we will replace this with the list of tasks that we have stored in our JSON file.

**Task**: Change the `index()` function that it loads the JSON file into a `item_dict` and returns a string of the usernames and tasks contained in the `item_dict`.

Now imagine you would like to create a good looking website. Of course, you could return the full source code for the HTML website. Something like this:

In [ ]:
    return """
           <!DOCTYPE html>
           <html lang="en">
           <head>
               <meta charset="UTF-8">
               <title>ToDo</title>
          </head>
          <body>
              
         </body>
         </html>
""".format(item_dict_output)

While this works, it will get ugly pretty quickly for large HTML pages. Enter the fantastic world of templates!

The idea is to store the HTML page in a separate file and simply replace certain parts of the file with dynamic content. Such as the content from the `item_dict`.


Before we start playing around with templates, we need a place where they can live first. Create a directory called `templates` next to your Flask Python file. Inside the templates directory create a file `index.html` with the following content:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>ToDo</title>
</head>
<body>

</body>
</html>

In order to render the template and send it to the browser Flask provides the `render_template` function. Import it first:

In [ ]:
from flask import Flask, request, jsonify, render_template
import json

Then simply call it with the name of the template as the parameter in your `return` statement in the `index()` function:

In [ ]:
return render_template('index.html')

If you now refresh your browser you will see an empty page. Let's fill it with the content from `item_dict`. The first task is to hand the todo items stored in `item_dict` over to the template. That is very simple to accomplish. Any additional keyword parameter in `render_template()` is made available in the template. So the following does the job nicely:

In [ ]:
return render_template('index.html', items=item_dict['items'])

The second task is to modify the template such that the todo items are rendered as valid HTML code. Let's start by displaying the first todo item only (that is `items[0]`). Flask uses a template engine/language called Jinja. In Jinja a variable wrapped in two curly braces is replaced with its value when the page is rendered. Try it by adding the following to the body of `index.html`:

In [ ]:
Username: {{ items[0]['username'] }}

**Task**: Refresh the page and check the result. Add the task of the first todo item as well. Wrap the jinja templates in `<ul><li></li></ul>` tags so they are rendered as a list. Add the second todo item.

Obviously, for multiple items/an unknown number of items this is approach does not really work. But Jinja provides a solution with the

`{% for %} {% endfor %}`

statement. Modify your `index.html` template such that it iterates over all todo items:

In [ ]:
<body>
    <ul>
    {% for item in items  %}
    <li>
    [See Task]
    </li>
    {% endfor  %}
    </ul>
</body>

**Task**: Fill in the part where it says `[See Task]` such that it prints the item's username and task for each todo item.

## Add a form

So far we have added data to the JSON file via calls to `http`. However, users of our webpage won't have access to this. Therefore, we will add a little form such that users can enter data. Simply add the following HTML code to your HTML file:

In [ ]:
<form action="/item/add" method="POST">
  Username:<br>
  <input type="text" name="username">
  <br>
  Task:<br>
  <input type="text" name="task">
  <br><br>
  <input type="submit" value="Submit">
</form>

Check out the beauty of our code. We already have the `/item/add` route accepting POST requests set up! This simple HTML form does exactly the same as our previous `http` requests on the command line: It sends the url-form-encoded data via a POST request to `/item/add`.

**Task**: Try adding a few entries and watch what happens.

Not happy that after clicking `Submit` the return value from `/item/add` is displayed? No problem just redirect the user back to the `index` page.

Add the following functions to the Flask `import` statements:

In [ ]:
redirect, url_for

And change the return statement to:

In [ ]:
return redirect(url_for('index'))

## Bonus: Make it beautiful

The current page looks a bit boring. Very 80's. Let's change that. We will use Bootstrap (http://getbootstrap.com/) for beautifying the page. Fortunately, there is a Flask extension for Bootstrap, which makes it very simple to add style.

Import the `Bootstrap` class from the extension:

In [ ]:
from flask_bootstrap import Bootstrap

Now tell the system to use Bootstrap by wrapping the application object like this:

In [ ]:
app = Flask(__name__)
Bootstrap(app)

We are almost done. The last thing left to do is to modify the HTML template file. Remove everything in your template file, except the code between the `<body></body>` tags (get rid of the `<body>` tags as well).

Add the following to the top of your HTML file:

In [ ]:
{% extends "bootstrap/base.html" %}
{% block title %}Todo{% endblock %}

Those are special Jinja statements that allow to extend an existing template with additional content. You could also refer to it as template inheritance. The `base.html` file defines so called `blocks` that you can use in order to ingest your content. The `title` block is one of them.

Another one is the `content` block. As you might have guessed by now, we simply have to wrap all your carefully hand crafted HTML code inside the following statements.

In [ ]:
{% block content %}

{% endblock %}

Refresh your Browser and the page looks a lot better.

## Bonus tasks

- Implement the `@app.route('/user/<username>')` route such that it shows only the tasks for the specified username
- Implement the `@app.route('/item/<int:item_id>')` route such that it shows the task given by the `item_id`